In [46]:
import warnings
warnings.filterwarnings('ignore')
import stanza
import re
import time
import string
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
nltk.download("stopwords")
nltk.download('punkt') 
nltk.download('wordnet')
geolocator = Nominatim(user_agent="my_app")
stanza.download('ru')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2022-07-27 18:08:52 INFO: Downloading default packages for language: ru (Russian)...
2022-07-27 18:08:53 INFO: File exists: C:\Users\User\stanza_resources\ru\default.zip.
2022-07-27 18:08:57 INFO: Finished downloading models and saved to C:\Users\User\stanza_resources.


In [34]:
txt="""В соответствии с постановлением Правительства Москвы от 16 ноября 2010 г. № 1019-ПП «О создании Градостроительно-земельной комиссии города Москвы», решением Градостроительно-земельной комиссии города Москвы (протокол от 6 февраля 2020 г. № 4) Правительство Москвы постановляет:
1. Внести изменения в постановление Правительства Москвы от 16 марта 2010 г. № 212-ПП «О дальнейшей реализации инвестиционного проекта по адресу: ул. Остоженка, д. 9/14» (в редакции постановлений Правительства Москвы от 23 августа 2011 г. № 380-ПП, от 28 февраля 2012 г. № 75-ПП, от 13 июня 2013 г. № 385-ПП, от 17 марта 2015 г. № 120-ПП, от 1 марта 2016 г. № 75-ПП, от 17 января 2017 г. № 6-ПП, от 17 апреля 2018 г. № 332-ПП):
1.1.  В пункте 1 постановления слова «до 17 августа 2019 г.» заменить словами «до 17 февраля 2022».
1.2. Дополнить постановление пунктом 1(1) в следующей редакции:
«1(1). Определить следующий порядок взаиморасчетов между ООО  «ИДИЛЬ ДЕВЕЛОПМЕНТ» и городом Москвой в части замены права требования на долю города в объекте инвестиционной деятельности по адресу: ул. Остоженка, д 9/14 на денежный эквивалент согласно Отчету об оценке от 24 декабря 2019 г. № И28-19.
1(1)1. ООО «ИДИЛЬ ДЕВЕЛОПМЕНТ» передает в счет выкупа права требования на долю города в собственность города Москвы в лице Департамента городского имущества города Москвы жилые помещения (квартиры) в срок до 24 августа 2020 г.
1(1)2. В случае невыполнения ООО «ИДИЛЬ ДЕВЕЛОПМЕНТ» в срок (п. 1(1)1 настоящего постановления) обязательства по передаче жилой площади Инвестор уплачивает в бюджет города Москвы денежные средства (п. 1(1) настоящего постановления) в срок до 31 августа 2020 г.».
1.3. В пункте 7 постановления слова «министра Правительства Москвы, руководителя Департамента экономической политики и развития города Москвы Ефимова В.В.» заменить словами «руководителя Департамента инвестиционной и промышленной политики города Москвы Прохорова А.В.».
2. Контроль за выполнением настоящего постановления возложить на руководителя Департамента инвестиционной и промышленной политики города Москвы Прохорова А.В.
"""

In [35]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    DatesExtractor,
    AddrExtractor,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)

# Find docs

In [36]:
def find_date(sent_text,start_text):
    matches = dates_extractor(sent_text)
    matches=list(matches)
    dates_pos=[(i.start+start_text,i.stop+start_text) for i in matches]
    dates=[i.fact.as_json for i in matches]
    dates=[f"{f.get('day')}.{f.get('month')}.{f.get('year')}" for f in dates]
    dates=dict(zip(dates_pos,dates))
    return dates

In [37]:
def find_quotes_text(sent_text,start_text):
    pairs={"'":"'",
      '"':'"',
      "<":">",
      "«":"»"}
    q_pos=[]
    for pos,char in enumerate(sent_text):
        if((char==sent_text[pos]) and (char in list(pairs.keys()))):
            start=pos
            sent_text=sent_text.replace(char,"(",1)
            end=sent_text.find(pairs[char])
            sent_text=sent_text.replace(pairs[char],")",1)
            q_pos.append((start,end))
        #print(start,end)
    q_pos_corr=[]
    q_pos_corr.append(q_pos[0])
    for q in q_pos[1:]:
        if (q[0]>q_pos_corr[-1][1]):
            q_pos_corr.append(q)
    q_pos_corr=[(q[0]+start_text,q[1]+start_text) for q in q_pos_corr] 
    return q_pos_corr

In [38]:
def find_near(start,stop,pos_list):
    pos_list=[(i[0],i[1]) for i in pos_list if (i[1]<start)]
    pos_min=[start-i[1] for i in pos_list]
    pos_min=pos_list[pos_min.index(min(pos_min))] 
    return pos_min

In [39]:
def find_docs(doc):
    nps_types=["конституция","закон","указ","постановление","акт"]
    doc_names={}
    for sent in doc.sents:
        key_words={}
        for token in sent.tokens:
            token.lemmatize(morph_vocab)
            #встречаются ключевые слова
            if (token.lemma in nps_types):
                key_words[(token.start,token.stop)]=token.lemma
        try:  
            names=find_quotes_text(sent.text,sent.start)
            dates=find_date(sent.text,sent.start)
            numbers=[pos for pos,char in enumerate(sent.text) if ((char =='№') or (char =="#"))]
            numbers=[(number,number+1+len(sent.text[number:].split(" ")[1])) for number in numbers]
            numbers=[(number[0]+sent.start,number[1]+sent.start) for number in numbers]
            for i in names:
                start=i[0]
                stop=i[1]
                doc_text=txt[start:stop+1]
                doc_number=find_near(start,stop,numbers)
                doc_number=txt[doc_number[0]:doc_number[1]]
                doc_date=find_near(start,stop,dates.keys())
                doc_date=dates[doc_date]
                doc_key=find_near(start,stop,key_words.keys())
                start=doc_key[0]
                doc_key=key_words[doc_key]
                #print(doc_key+" "+doc_date+" "+doc_number+" "+doc_text)
                doc_names[doc_key+" "+doc_date+" "+doc_number+" "+doc_text]={
                    "doc_key": doc_key,
                    "doc_date":doc_date,
                    "doc_num":doc_number,
                    "doc_text":doc_text,
                    "doc_pos":(start,stop+1)
                }
        except:
            pass
    return doc_names

In [40]:
def find_consultant(text):
    """
    find doc in consultant database
    """
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get('http://www.consultant.ru/')
    search = browser.find_element_by_name('q')
    search.send_keys(text)
    search.send_keys(Keys.RETURN)
    #search.click()
    time.sleep(3)
    html = browser.page_source
    soup = BeautifulSoup(html)
    elements=soup.findAll("a",class_="search-results__link")
    el_set=dict()
    i=0
    for el in elements:
        text=el.find("p",class_="search-results__text").text.replace("\n","")
        text_d=el.find("p",class_="search-results__link-inherit").text.replace("\n","")
        remark= el.find("p",attrs={'class': re.compile('search-results__status*')})
        if(remark):
            remark=re.sub(' +', ' ',remark.text.replace("\n",""))
        else:
            remark=""
        el_set[i]={"text":text+" "+text_d,"remark":remark}
        i+=1
    return el_set

In [47]:
def docs_seacher(txt):
    """
    get detailed info about all docs in txt
    """
    doc = Doc(txt)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc_names=find_docs(doc)
    for name in doc_names:
        search_text=doc_names[name]["doc_pos"]
        search_text=txt[search_text[0]:search_text[1]]
        search_text=find_consultant(search_text)
        correct=False
        for i in search_text:
            dates=list(find_date(search_text[i]["text"],0).values())
            numbers=[pos for pos,char in enumerate(search_text[i]["text"]) if (char =='N')]
            numbers=[search_text[i]["text"][number:number+1+len(search_text[i]["text"][number:].split(" ")[1])] for number in numbers]
            numbers=[num.split()[1] for num in numbers ]
            if doc_names[name]["doc_date"] in dates:
                if doc_names[name]["doc_num"].split()[1] in numbers:
                    correct=True
                    doc_names[name]["correct"]=correct
                    if search_text[i]["remark"]:
                        status=search_text[i]["remark"]
                    else:
                        status="Действует"
                    doc_names[name]["status"]=status
        if (correct==False):
            doc_names[name]["correct"]=correct
    return doc_names

In [ ]:
doc_names=docs_seacher(txt)

In [49]:
doc_names

{'постановление 16.11.2010 № 1019-П «О создании Градостроительно-земельной комиссии города Москвы»': {'doc_key': 'постановление',
  'doc_date': '16.11.2010',
  'doc_num': '№ 1019-П',
  'doc_text': '«О создании Градостроительно-земельной комиссии города Москвы»',
  'doc_pos': (17, 146),
  'correct': True,
  'status': 'Действует'},
 'постановление 16.3.2010 № 212-П «О дальнейшей реализации инвестиционного проекта по адресу: ул. Остоженка, д. 9/14»': {'doc_key': 'постановление',
  'doc_date': '16.3.2010',
  'doc_num': '№ 212-П',
  'doc_text': '«О дальнейшей реализации инвестиционного проекта по адресу: ул. Остоженка, д. 9/14»',
  'doc_pos': (300, 447),
  'correct': True,
  'status': 'Действует'}}

# Find addresses

In [72]:
def find_loc(sent,city_comp):
    add=False
    one_address=[]
    addresses=[]
    for word in sent:
        word=re.sub(r'[!@"$;%^:&?*()<>~`|«»#№]', '', word)
        word=word.rstrip()
        if (add):
            if(word!=""):
                one_address.append(word)
            else:
                one_address=[]
        if word in city_comp:
            add=True
        else:
            if (len(one_address)>1 and add==False):         
                addresses.append(' '.join(one_address))
                one_address=[]
            add=False
    return addresses   
def find_loc_txt(txt):
    """
    find any location in txt
    """
    city_comp={'ул.',"улица",'стр.',"строение",'дом ','д.',"р-н",' район','владение','корп',"корпус","область","обл.",
           "г.","город ","пос.","поселок","пр.","проезд","пер.","переулок","наб.","набережная","пл.","площадь",
           "б-р","бульвар","линия","ш.","шоссе"}
    doc = Doc(txt)
    doc.segment(segmenter)
    sents=[d.text for d in doc.sents]
    addresses=[]
    for sent in sents:
        sent=sent.split()
        addresses.extend(find_loc(sent,city_comp))
    return addresses
    

In [73]:
addresses=find_loc_txt(txt)
addresses=list(set(addresses))
address_check={}#словарь адрес- существует/нет
for address in addresses:
    """
    check is address really exist using geopy
    """
    location = geolocator.geocode(address)
    if location:
        address_check[address]=True
    else:
        address_check[address]=False

In [75]:
address_check

{'Остоженка, 9/14': True}

# Find organization

In [195]:
russian_stopwords = stopwords.words("russian")
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words=russian_stopwords)

In [ ]:
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map))

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [117]:
def companies_info(names,texts):
    companies={}
    for i in range(len(names)):
        loc=texts[i].split(", ОГРН")[0]
        loc=loc.lower()
        try:
           #boss=texts[i].split("ДИРЕКТОР")[1].split(",")[0]
           boss=texts[i].split("КПП:")[1].split(": ")[1].split(",")[0].split("\n")[0]
        except:
            print(texts[i])
            boss=False
        try:
            end_data=texts[i].split("Дата прекращения деятельности: ")[1].split("\n")[0]
        except:
            end_data=False
        companies[names[i].lower()]={"loc":loc,"boss":boss,"end_data":end_data}
    return companies
def get_info_org(org):
    """organization information scraping """
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get('https://egrul.nalog.ru/index.html')
    search = browser.find_element_by_name('query')
    search.send_keys(org)
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    elems=search.find_elements_by_xpath("//div[@class='res-row']//div[@class='res-caption']")
    names=[el.text for el in elems if (el.text)]
    elems=search.find_elements_by_xpath("//div[@class='res-row']//div[@class='res-line']")
    texts=[el.text for el in elems if (el.text)]
    browser.quit()
    return names,texts

In [118]:
def find_around(start,stop,pos_list):
    #pos_list=[(i[0],i[1]) for i in pos_list if (i[1]<start)]
    pos_min=[min(abs(start-i[1]),abs(stop-i[0])) for i in pos_list]
    pos_min=pos_list[pos_min.index(min(pos_min))] 
    return pos_min

In [ ]:
doc = Doc(txt)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)
locs={}
for span in doc.spans:
    span.normalize(morph_vocab)
    if span.type == "LOC":
        locs[(span.start,span.stop)]=span.text
orgs={}
for span in doc.spans:
    span.normalize(morph_vocab)
    if span.type == "ORG":
        orgs[span.normal]={"pos":(span.start,span.stop),
                          "loc":locs[find_around(span.start,span.stop,list(locs.keys()))].lower()
                          }
for org in list(orgs.keys()):
    name_org=org.lower().split('«')[-1]
    name_org=name_org.split("»")[0]
    location=orgs[org]["loc"]
    location=morph.parse(location)[0].normal_form
    names,texts=get_info_org(org)
    companies=companies_info(names,texts)
    new_companies={}
    for comp in companies:
        if location in companies[comp]['loc']:
            new_companies[comp]=companies[comp]
        company={}
    for c in new_companies:
        if name_org in c:
            for k in new_companies[c]:
                orgs[org][k]=new_companies[c][k]
            break

In [249]:
orgs

{'Правительство': {'pos': (1705, 1718), 'loc': 'москвы'},
 'Градостроительно-земельной комиссия': {'pos': (157, 192), 'loc': 'москвы'},
 'ООО  «ИДИЛЬ ДЕВЕЛОПМЕНТ': {'pos': (927, 950),
  'loc': '115184, россия, г. москва, вн.тер.г. муниципальный округ замоскворечье, большой овчинниковский пер., д. 16, этаж 5, помещ. 525а',
  'boss': 'Качановский Дмитрий Евгеньевич',
  'end_data': False},
 'ООО «ИДИЛЬ ДЕВЕЛОПМЕНТ': {'pos': (1425, 1447),
  'loc': '115184, россия, г. москва, вн.тер.г. муниципальный округ замоскворечье, большой овчинниковский пер., д. 16, этаж 5, помещ. 525а',
  'boss': 'Качановский Дмитрий Евгеньевич',
  'end_data': False},
 'Департамент городского имущества': {'pos': (1292, 1325),
  'loc': '123112, город москва, 1-й красногвардейский проезд, дом 21, строение 1',
  'boss': 'Дегтярева Ирина Ивановна',
  'end_data': False},
 'Департамент экономической политики и развития': {'pos': (1740, 1786),
  'loc': '125032, город москва, тверская улица, дом 13',
  'boss': 'Пуртов Кирилл